# MATH&ML-6 Математический анализ в контексте задачи оптимизации. Часть 3
###  Содержание <a class="anchor" id=0></a>

- [1. О чём лекция](#2)
- [2. Градиентный спуск: применения и классификации](#2)
- [3. Метод Ньютона](#3)
- [4. Квазиньютоновские методы](#4)
- [5. Линейное программирование](#5)
- [6. Практика: Линейное программирование](#6)
- [7. Практика: Дополнительные методы оптимизации](#7)
- [8. Итоги](#8)


# 1. О чём лекция <a class="anchor" id=1></a>

[к содержанию](#0)

* рассмотрим, какие вариации существуют у уже известного вам алгоритма градиентного спуска, и узнаем, в чём суть **обратного распространения ошибки**;

* познакомимся с **методом Ньютона** и **квазиньютоновскими методами BFGS** и **L-BFGS**;

* разберём область применения задач **линейного программирования** и попрактикуемся в их решении;

* узнаем, что такое **метод отжига** и **метод координатного спуска**.

>Из предыдущего модуля вы помните, что в каких-то методах оптимизации мы использовали лишь значение функции, где-то — считали градиент, а где-то — находили матрицы производных. Эти особенности дают возможность поделить все алгоритмы на **три группы**:
>
>* методы нулевого порядка (их работа основана на оценке значений самой целевой функции в разных точках);
>
>* методы первого порядка (при работе они используют первые производные в дополнение к информации о значении функции);
>
>* методы второго порядка (для них необходимо оценивать и значение функции, и значение градиента, и гессиан (матрицу Гессе)).

**Примечание**. Иногда в литературе можно встретить термины «оракул первого порядка» или «оракул нулевого порядка». Так обозначают компоненты алгоритма, которые находят информацию на каждом шаге для метода соответствующего порядка.

В данном модуле мы будем рассматривать методы разных порядков и практически в каждом юните будем практиковаться в их применении. В конце мы обобщим полученные знания и сравним, какие методы и в каких случаях показывают наилучшие результаты.

→ Важно отметить, что некоторые алгоритмы оптимизации мы пока не сможем использовать по прямому назначению: мы опробуем их на известных вам моделях линейной или логистической регрессии или на функциях одной или нескольких переменных, но основная сфера их применения — это искусственные нейронные сети. Тем не менее, понимание всего пула алгоритмов поможет вам в будущем без проблем подбирать необходимый метод вне зависимости от того, задачу какой сложности вы решаете: минимизируете простейшую функцию или обучаете нейронную сеть со сложной архитектурой.

***

**Задание 1.1**

Пусть прибыль в вашей компании выражается следующей функцией, которая зависит от параметра  — количества производимых товаров:

$f(x)=-x^{4}+6 x^{3}-4 x^{2}+80$

Найдите максимально возможную прибыль, которую вы можете получить, варьируя количество произведённых товаров.

In [29]:
from sympy import Symbol, Eq, diff, solve

x = Symbol('x')
fun = -x**4+6*x**3-4*x**2+80
df = diff(fun)
sol = solve(Eq(df,0))

print(f'x={sol[0]} => f={fun.subs(x,sol[0])} \nx={sol[1]} => f={fun.subs(x,sol[1])} \nx={sol[2]} => f={fun.subs(x,sol[2])}')


x=0 => f=80 
x=1/2 => f=1275/16 
x=4 => f=144


**Задание 1.4**

Допустим, вы хотите произвести некоторое количество товара, которое зависит от часов работы двух ключевых сотрудников следующим образом:

$f(x, y)=x^{2}+2 y^{2}$

Однако вы можете оплатить этим сотрудникам не более 20 часов работы.

Какое наибольшее количество товаров вы сможете произвести в таком случае?

In [46]:
from sympy import symbols, Eq, solve, diff

x, y, z = symbols('x y z') 
f = x**2+2*y**2 + z*(x+y-20) # запишем функцию Лагранжа
f_1 = x**2+2*y**2 # запишем исходную функцию

f_x = diff(f,x) # найдём частные производные от ф-и Ларанжа

f_y = diff(f,y)
f_z = diff(f,z)

sol = solve([Eq(f_x,0), Eq(f_y,0), Eq(f_z,0)], [x, y, z]) # разрешим систему уравнений собраную из частных производных
print(f'Первый работник: {round(sol[x])} ч/часов\nВоторой работник: {round(sol[y])} ч/часов')
print(f'Кол-во товаров: {f_1.subs([(x,round(sol[x])),(y,round(sol[y]))])} шт')

Первый работник: 13 ч/часов
Воторой работник: 7 ч/часов
Кол-во товаров: 267 шт


# 2. Градиентный спуск: применения и классификации <a class="anchor" id=2></a>

[к содержанию](#0)

Чтобы лучше понимать, какую роль играют методы оптимизации в DS и почему их так много, важно понимать основные **принципы работы нейронных сетей**. Также в современных статьях и обзорах разных методов оптимизации они практически всегда рассматриваются в контексте применения в нейронных сетях.

Процесс обучения человеческого мозга очень сложен, и наука пока не может дать достаточно подробный ответ на вопрос о том, как мы получаем и усваиваем знания, как принимаем решения. Однако той информации, которую уже удалось получить, оказалось достаточно, чтобы по аналогии создать модели искусственных нейронных сетей.

Люди учатся через пробы и ошибки, через процесс **синаптической пластичности**. Точно так же, как связи в мозге укрепляются и формируются по мере того, как мы переживаем новые события, мы обучаем искусственные нейронные сети, вычисляя ошибки нейросетевых предсказаний и усиливая или ослабляя внутренние связи между нейронами на основе этих ошибок.

>**Синаптическая пластичность** — это понятие, которое используется для описания того, как формируются и укрепляются нейронные связи после получения новой информации.
Для лучшего понимания удобно воспринимать нейронную сеть как функцию, которая принимает входные данные для получения итогового прогноза. Переменными этой функции являются параметры, или веса, нейрона.

Следовательно, ключевым моментом для решения задачи, которую мы ставим для нейронной сети, будет **корректировка значений весов** таким образом, чтобы они аппроксимировали или наилучшим образом представляли набор данных.

На изображении ниже показана простая нейронная сеть, которая получает входные данные $(X_1, \ X_2, \  X_3, \ X_n)$. Эти входные данные передаются нейронам в слое, содержащем веса $(W_1, \ W_2, \  W_3, \ W_n)$. Входные данные и веса подвергаются операции умножения, и результат суммируется с помощью специальной функции, а функция активации регулирует конечный результат модели.

<img src=m6_img1.png width=700>

Для того чтобы оценить, насколько эффективно работает наша нейронная сеть, необходим показатель оценки разницы между предсказанием нейронной сети и фактическим значением целевой функции, позволяющий корректировать параметры сети так, чтобы разница между прогнозом и реальностью была как можно меньше. В `Data Science` эту разницу часто называют **функцией стоимости**.

Ниже представлена модель работы простейшей нейронной сети:

<img src=m6_img2.png width=700>

На этом изображении мы можем видеть модель простой нейронной сети из плотно связанных нейронов, которая классифицирует рукописные представления цифр 0, 1, 2, 3. Каждый нейрон в выходном слое соответствует цифре. Чем выше активация соединения с нейроном, тем выше вероятность, выдаваемая нейроном. Вероятность соответствует вероятности того, что цифра, переданная вперёд по сети, связана с активированным нейроном.

Когда цифра 3 передаётся через сеть, мы ожидаем, что соединения (представленные на диаграмме стрелками), ответственные за классификацию этой цифры, будут иметь более высокую активацию, что приводит к более высокой вероятности для выходного нейрона, связанного с цифрой 3.

>Если объяснить происходящее более простыми словами, то механизм работы примерно такой:
>
>1. На первом слое мы выделяем из цифры какие-то первичные признаки (округлости, палочки).
>
>2. На втором слое мы выделяем уже какие-то паттерны (фрагменты цифры).
>
>3. На третьем слое паттерны складываются в целую цифру и мы можем предсказать результат.
>
>Каждый раз элементы, более похожие на элементы цифры 3, дают более высокую активацию.

За активацию нейрона отвечает несколько компонентов, и мы должны менять их в процессе обучения нашей нейронной сети, повышая качество её предсказания.

Допустим, мы используем уже хорошо известную вам среднеквадратичную ошибку как меру для оценки качества модели. В этом случае мы каждый раз вычисляем её, получаем результат и отправляем его обратно для коррекции весов в сети. Здесь как раз и появляется понятие **обратного распространения ошибки**.

Обратное распространение (`backpropagation`) — это механизм, с помощью которого компоненты, влияющие на итоговый результат, итеративно корректируются для уменьшения функции стоимости.

Важнейшим математическим процессом, связанным с обратным распространением, является вычисление производных. Операции обратного распространения вычисляют частную производную функции стоимости по отношению к весам и активациям предыдущего слоя, чтобы определить, какие значения влияют на градиент функции стоимости.

>В процессе минимизации  функции ошибки мы постоянно вычисляем значение градиентов и таким образом приходим к локальному минимуму. На каждом этапе обучения нейронной сети её веса пересчитываются с помощью найденного градиента, причём скорость обучения (или, мы уже называли её ранее, темп обучения или шаг градиентного спуска) определяет коэффициент, с которым вносятся изменения в значения весов. Это повторяется на каждом шаге обучения нейронной сети. Нашей целью является постоянное приближение к локальному минимуму.

Принцип обратного распространения ошибки заключается в том, что сначала мы устанавливаем какие-то случайные веса (параметры) для модели, находим итоговую ошибку и движемся по сети обратно, корректируя веса (для этого вычисляем частные производные, т. е. градиент).

Данный процесс выглядит следующим образом:

<img src=m6_gif1.gif>

Распространение ошибок и использование частных производных для корректировки весов происходит до тех пор, пока не будут скорректированы все параметры в сети вплоть до первого слоя.

>Если вам хочется больше узнать про механизм обратного распространения ошибки, рекомендуем обратиться к [этой статье](https://brilliant.org/wiki/backpropagation/).

Как вы можете видеть, даже в самом простом примере нейронной сети много переменных и действий, которые с ними происходят. Из-за этого ландшафт функции потерь для нейронных сетей становится очень сложным. К примеру, ландшафт для нейронной сети с 56 слоями может выглядеть так:

<img src=m6_img3.png width=400>

В предыдущем модуле мы без проблем решили задачу с помощью классического градиентного спуска, и результат совпал с итогом, полученным с помощью реализации алгоритма в стандартной библиотеке `Python`. Но функция потерь там была совершенно другой. Когда дело доходит до искусственных нейронных сетей, алгоритмы оптимизации сталкиваются с множеством проблем.

>Основные проблемы при реализации градиентного спуска:
>
>* Классический градиентный спуск **склонен застревать** в точках локального минимума и даже в седловых точках, словом — везде, где градиент равен нулю. Это мешает найти глобальный минимум.
>
>* Обычно у оптимизируемой функции **очень сложный ландшафт**: где-то она совсем пологая, где-то более крутой обрыв. В таких ситуациях градиентный спуск показывает не лучшие результаты. Так происходит потому, что в алгоритме градиентного спуска фиксированный шаг, а нам в идеале хотелось бы его изменять в зависимости от формы функции прямо в процессе обучения.
>
>* Много проблем возникает из-за **темпа обучения**: при низком алгоритм сходится невероятно медленно, при быстром — «пролетает» мимо минимумов.
>
>* При обучении градиентного спуска координаты в некоторых измерениях могут редко изменяться, что приводит к плохой обобщающей способности алгоритма. Можно попытаться придать каждого признаку бόльшую важность, но в таком случае есть серьёзный **риск переобучить модель**.

Тем не менее, нельзя отрицать, что градиентный спуск — невероятно эффективный и популярный алгоритм. Допустим, он прекрасно подходит для минимизации среднеквадратичной ошибки в случае решения задачи регрессии. Однако в силу его несовершенств, которые очень явно проявляются в сложных моделях (например, в нейронных сетях), были созданы некоторые его модификации, которые позволяют решать задачи с большей результативностью.

>Обычно выделяют три основных вариации градиентного спуска:
>
>* `Batch Gradient Descent`;
>
>* `Stochastic Gradient Descent`;
>
>* `Mini-batch Gradient Descent`.
>

Далее мы рассмотрим все эти модификации и обсудим их различия. Важно понимать, что, несмотря на то что про их применение часто говорят именно в контексте нейронных сетей, они прекрасно подходят и для использования с обычными методами машинного обучения. К примеру, для стандартной линейной регрессии также подходит и обычный градиентный спуск.

## BATCH GRADIENT DESCENT

Первая вариация — это `Batch Gradient Descent`. По-русски её называют **пакетным градиентным спуском** (Пакетным его называют по той причине, что он использует всю выборку (весь пакет) на каждом шаге, для того чтобы получить результат.), или *ванильным градиентным спуском* (хотя англоязычную вариацию Vanilla Gradient Descent чаще не переводят). По сути, это и есть классический градиентный спуск, который мы с вами рассматривали в предыдущем модуле.

<img src=m6_img4.png width=700>

Часто в различных источниках шаг `Batch Gradient Descent` записывается в следующих обозначениях:

$\theta=\theta-\eta \cdot \nabla_{\theta} J(\theta)$

>**Примечание**. На самом деле совершенно не важно, какими буквами выражать те или иные объекты, но мы будем использовать в этом модуле обозначения, которые часто встречаются в научных статьях и различной литературе.

Здесь $\theta$ — вектор с параметрами функции, $\eta$ — шаг градиента, $\nabla_{\theta} J(\theta)$ — градиент функции, найденный по её параметрам.

Таким образом, на каждом шаге градиентный спуск находит направление наискорейшего убывания функции и движется чётко по нему. Поэтому для несложных функций он достаточно быстро сходится. Его обучение можно изобразить следующим образом:

<img src=m6_img5.png width=700>

Такой градиентный спуск достаточно хорошо работает, если мы рассматриваем выпуклые или относительно гладкие функции ошибки. Такие, к примеру, мы можем наблюдать у **линейной** или **логистической регрессии**. Мы обсуждали, что градиентный спуск не умеет находить глобальный минимум среди прочих, но, например, для выпуклой функции он может это сделать. Поэтому с выпуклыми функциями (допустим, со среднеквадратичной ошибкой для линейной регрессии) нам удобнее всего использовать именно его.

>Но всё усложняется, когда мы хотим применить его при обучении нейронных сетей. Как уже говорилось, у таких моделей функция потерь имеет много локальных экстремумов, в каждом из которых градиентный спуск может застрять. Когда данных очень много, а оптимизируемая функция очень сложная, данный алгоритм применять затруднительно, так как поиск градиента по всем наблюдениям делает задачу очень затратной в плане вычислительных ресурсов.

## STOCHASTIC GRADIENT DESCENT

Представим, что мы реализуем градиентный спуск для набора данных объёмом 10 000 наблюдений и у нас десять переменных. Среднеквадратичную ошибку считаем по всем точкам, то есть для 10 000 наблюдений. Производную необходимо посчитать по каждому параметру, поэтому фактически за каждую итерацию мы будем выполнять не менее 100 000 вычислений. И если, допустим, у нас 1000 итераций, то нам нужно 100000*1000=100000000 вычислений. Это довольно много, поэтому градиентный спуск на сложных моделях и при использовании больших наборов данных работает крайне долго.

Чтобы преодолеть эту проблему, придумали **стохастический градиентный спуск**. Слово *«стохастический»* можно воспринимать как синоним слова *«случайный»*. Где же при использовании градиентного спуска может возникнуть случайность? При выборе данных. При реализации стохастического спуска вычисляются градиенты не для всей выборки, а только для случайно выбранной единственной точки.

<img src=m6_img6.png width=700>

Это значительно сокращает вычислительные затраты.

В виде формулы это можно записать следующим образом:

$\theta=\theta-\eta \cdot \nabla_{\theta} J\left(\theta ; x^{(i)} ; y^{(i)}\right)$

На визуальном представлении ниже можно увидеть, что стохастический спуск создаёт много колебаний при сходимости. Это происходит как раз за счёт того, что берётся не вся выборка, а только один объект, и между объектами может быть достаточно большая разница. Чем меньше выборка, тем меньше стабильности при реализации.

<img src=m6_img7.png width=700>

>Стохастический градиентный спуск **очень часто используется в нейронных сетях** и сокращает время машинных вычислений, одновременно повышая сложность и производительность крупномасштабных задач.

## MINI-BATCH GRADIENT DESCENT

Третья вариация градиентного спуска — `Mini-batch Gradient Descent`. Также можно называть его **мини-пакетным градиентным спуском**. По сути, эта модификация сочетает в себе лучшее от классической реализации и стохастического варианта. На данный момент это наиболее популярная реализация градиентного спуска, которая используется в глубоком обучении (т. е. в обучении нейронных сетей).

В ходе обучения модели с помощью мини-пакетного градиентного спуска обучающая выборка разбивается на пакеты (**батчи**), для которых рассчитывается ошибка модели и пересчитываются веса.

<img src=m6_img8.png width=700>

>То есть, с одной стороны, мы используем все преимущества обычного градиентного спуска, а с другой — уменьшаем сложность вычислений и повышаем их скорость по аналогии со стохастическим спуском. Кроме того, алгоритм работает ещё быстрее за счёт возможности применения векторизованных вычислений.

Формализовать это можно следующим образом:

$\theta=\theta-\eta \cdot \nabla_{\theta} J\left(\theta ; x^{(i: i+n)} ; y^{(i: i+n)}\right)$

Как показывает визуализация ниже, амплитуда колебаний при сходимости алгоритма больше, чем в классическом градиентном спуске, но меньше, чем в стохастическом:

<img src=m6_img9.png width=700>

|BATCH GRADIENT DESCENT|STOCHASTIC GRADIENT DESCENT|MINI-BATCH GRADIENT DESCENT|
| - | - | - |
|Рассматриваются все обучающие данные|Рассматривает только один объект|Рассматривается подвыборка|
|Затрачивает много времени на работу|Работает быстрее пакетного|Работает быстрее двух других|
|Плавное обновление параметров модели|Сильные колебания в обновлении параметров модели|Колебания зависят от размера подвыборки (увеличиваются с уменьшением её объема)|

**Задание 2.7**

Давайте потренируемся применять стохастический градиентный спуск для решения задачи линейной регрессии. Мы уже рассмотрели его реализацию «с нуля», однако для решения практических задач можно использовать готовые библиотеки.

Загрузите стандартный датасет об алмазах из библиотеки `Seaborn`:

Разделите выборку на обучающую и тестовую (объём тестовой возьмите равным 0.33), значение `random_state` должно быть равно 42.

Теперь реализуйте алгоритм линейной регрессии со стохастическим градиентным спуском (класс `SGDRegressor`). Отберите с помощью `GridSearchCV` оптимальные параметры по следующей сетке:

In [50]:
import seaborn as sns
import pandas as pd
import numpy as np

df = pd.read_csv('diamonds.csv')
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [2]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import pandas as pd
import numpy as np

df = pd.read_csv('diamonds.csv')

df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)
df = pd.get_dummies(df, drop_first=True)

df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

X_cols = [col for col in df.columns if col!='price']
X = df[X_cols]
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

parameters = {
    "loss": ["squared_error", "epsilon_insensitive"],
    "penalty": ["elasticnet"],
    "alpha": np.logspace(-3, 3, 15),
    "l1_ratio": np.linspace(0, 1, 11),
    "max_iter": np.logspace(0, 3, 10).astype(int),
    "random_state": [42],
    "learning_rate": ["constant"],
    "eta0": np.logspace(-4, -1, 4)
}

sgd = SGDRegressor(random_state=42)
sgd_cv = GridSearchCV(estimator=sgd, param_grid=parameters, n_jobs=-1)
sgd_cv.fit(X_train, y_train)

print(sgd_cv.best_params_)

sgd = SGDRegressor(**sgd_cv.best_params_)

sgd.fit(X_train, y_train)
sgd.score(X_train, y_train) # r2
ls = sgd.predict(X_test)

round(mean_squared_error(y_test, ls), 3)

{'alpha': 0.001, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'squared_error', 'max_iter': 21, 'penalty': 'elasticnet', 'random_state': 42}


0.044

## *БОНУС: АЛГОРИТМЫ, ОСНОВАННЫЕ НА ГРАДИЕНТНОМ СПУСКЕ

Градиентный спуск настолько популярен и хорошо применим для решения различных задач, что послужил основой множества дополнительных методов. Поговорим про некоторые из них.

Иногда в данных присутствуют очень редко встречающиеся входные параметры.

>Например, если мы классифицируем письма на «Спам» и «Не спам», таким параметром может быть очень специфическое слово, которое встречается в спаме намного реже других слов-индикаторов. Или, если мы говорим о распознавании изображений, это может быть какая-то очень редкая характеристика объекта.

В таком случае нам хотелось бы иметь для каждого параметра свою скорость обучения: чтобы для часто встречающихся она была низкой (для более точной настройки), а для совсем редких — высокой (это повысит скорость сходимости). То есть нам очень важно уметь обновлять параметры модели, учитывая то, насколько типичные и значимые признаки они кодируют.

Решение этой задачи предложено в рамках алгоритма `AdaGrad` (его название обозначает, что это адаптированный градиентный спуск). В нём обновления происходят по следующему принципу:

$G_t = G_t + g^2_t$

$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{G_t + \epsilon}} g_t$



Здесь мы храним сумму квадратов градиентов для каждого параметра. Таким образом, параметры, которые сильно обновляются каждый раз, начинают обновляться слабее. Скорость обучения в таком алгоритме будет постоянно затухать. Мы будем начинать с больших шагов, и с приближением к точке минимума шаги будут уменьшаться — это улучшит скорость сходимости.

>Данный алгоритм достаточно популярен и работает лучше стохастического градиентного спуска. Его использует и компания Google в своих алгоритмах классификации изображений.

Однако снижение скорости обучения в `AdaGrad` иногда происходит слишком радикально, и она практически обнуляется. Чтобы решить эту проблему, были созданы алгоритмы `RMSProp`, `AdaDelta`, `Adam` и некоторые другие.

Если вам интересно подробнее узнать о перечисленных алгоритмах и окунуться в процесс оптимизации нейронных сетей, рекомендуем обратиться к следующим статьям:

* ["An overview of gradient descent optimization algorithms"](https://ruder.io/optimizing-gradient-descent/index.html#adagrad)

* [«Методы оптимизации нейронных сетей»](https://habr.com/ru/post/318970/)

# 3. Метод Ньютона <a class="anchor" id=3></a>

[к содержанию](#0)

Метод Ньютона используется во многих алгоритмах машинного обучения. Часто в литературе его сравнивают с градиентным спуском, так как два этих алгоритма очень популярны. Вы уже сталкивались с методом Ньютона, но не знали об этом.

В [документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) для функции `LogisticRegression` из библиотеки `scikit-learn` представлено пять вариантов алгоритмов оптимизации, которые можно использовать при обучении модели:

* 'newton-cg';

* 'lbfgs';

* 'liblinear';

*  'sag';

*  'saga'.

Последние два являются вариациями стохастического градиентного спуска (а значит вам уже понятен принцип их работы), а с первыми тремя нам только предстоит познакомиться. В этом юните мы рассмотрим алгоритм `'newton-cg'`, в следующем — `'lbfgs'`, а в седьмом юните — `'liblinear'`. Вы будете понимать суть всех методов, представленных в самой популярной библиотеке для машинного обучения, и выбирать подходящий, исходя из особенностей поставленной задачи.

Начнём с метода Ньютона. Этот алгоритм работает быстрее, чем градиентный спуск, и тратит меньше времени для достижения минимума, однако у него есть и определённые недостатки, о которых мы поговорим позже.

Метод Ньютона изначально появился как метод решения уравнений вида $f(x)=0$.

Проиллюстрируем принцип его работы геометрически. Пусть у нас есть график некоторой функции. Проведём к нему касательную в точке $x_n$.

<img src=m6_img10.png>



Тогда эта касательная имеет наклон, равный $f'(x_n)$, и проходит через точку $x_n, f(x_n)$. В таком случае мы можем сказать, что уравнение этой касательной: $y = f'(x_n) (x - x_n) + f(x_n)$.

Так как нам необходимо решить уравнение, то нужно попасть в такую точку $x_{n+1}$, чтобы в ней значение координаты по оси ординат было нулевым, то есть в точку с координатами $x =  x_{n+1}$ и $y=0$.

Подставим это в наше уравнение касательной:

$y=f^{\prime}\left(x_{n}\right)\left(x-x_{n}\right)+f\left(x_{n}\right)$

$x=x_{n+1}, y=0$

$0=f^{\prime}\left(x_{n}\right) \cdot\left(x_{n+1}-x_{n}\right)+f\left(x_{n}\right)$

$f^{\prime}\left(x_{n}\right) \cdot\left(x_{n+1}-x_{n}\right)=-f\left(x_{n}\right)$

$x_{n+1}-x_{n}=-\frac{f\left(x_{n}\right)}{f^{\prime}\left(x_{n}\right)}$

$x_{n+1}=x_{n}{-\frac{f\left(x_{n}\right)}{f^{\prime}\left(x_{n}\right)}}$

Можно посмотреть на это и в анимации: 

<img src=m6_gif2.gif>

Мы видим, как для $x$ вычисляется $f(x)$, строится касательная, и в точке пересечения касательной с осью $Ox$ строится новая точка, к которой также строится касательная, и так далее. Математически доказано, что таким образом $x_i$ приближается к значению, где $f(x)=0$. 

Формально первый  шаг этого алгоритма мы можем записать следующим образом:

$x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}$

Все остальные шаги можно обобщить с помощью следующей зависимости:

$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$

Шаги могут повторяться сколько угодно раз до достижения необходимой точности.

Давайте посмотрим, как с использованием этого метода можно решить уравнение ↓

**Пример № 1**

Найти корень уравнения $x^2 - 4x - 7 = 0$, который находится рядом с точкой $x=5$, с точностью до тысячных.

Функция: $f(x) = x^2 - 4x - 7 = 0$

Начальная точка: $x_0 = 5$

Производная для функции: $f'(x) = 2x - 4$

Начнём поочерёдно совершать шаги и переходить в следующие точки, используя формулу, которую мы рассмотрели ранее:

$x_1 = 5 - \frac{5^2 - 4 \times 5 - 7}{2 \times 5 - 4} = 5 - \frac{(-2)}{6}  = \frac{16}{3} \approx 5.33333$

$x_2 = \frac{16}{3} - \frac{(\frac{16}{3})^2 - 4 (\frac{16}{3}) - 7}{2 (\frac{16}{3}) - 4} = \frac{16}{3} - \frac{\frac{1}{9}}{\frac{20}{3}} = \frac{16}{3} - \frac{1}{60} = \frac{319}{60} \approx 5.31667$

$x_3 = \frac{319}{60} - \frac{(\frac{319}{60})^2 - 4 (\frac{319}{60}) - 7}{2 (\frac{319}{60}) - 4} = \frac{319}{60} - \frac{\frac{1}{3600}}{\frac{398}{60}} \approx 5.31662$

Мы видим, что в последних двух точках мы уже находимся примерно **в одном и том же месте**. На всякий случай проверим ещё одну:

$x_4 = 5.31662 - \frac{(5.31662)^2 - 4 (5.31662) - 7}{2 (5.31662) - 4} = 5.31662$

Действительно, мы нашли точку с точностью до тысячных, в которой останемся — это и будет нашим ответом.

**Пример № 2**

Найти корни сложного полинома $f(x) = 6x^5 - 5x^4 - 4x^3 + 3x^2$.

Как мы знаем, к сожалению, для полинома пятой степени нет формулы поиска корней, поэтому будем использовать численные методы. В этих случаях приходится прибегать к числовому линейному приближению.

Ниже представлен график нашего полинома. У него три корня: в точках 0, 1 и где-то между ними. Как найти третий корень?

<img src=m6_img11.png width=400>

В методе Ньютона мы берём случайную точку $x_0$, а затем проводим касательную в ней. Точка $x_1$, где эта касательная пересекает ось абсцисс, станет нашим следующим предположением. Так что теперь мы  строим уже касательную в этой точке, и так далее . Мы продолжаем до тех пор, пока не достигнем необходимой точности. В целом, мы можем сделать приближение настолько близким к нулю, насколько хотим.

**Задание 3.1**

Найдите третий корень полинома $f(x) = 6x^5 - 5x^4 - 4x^3 + 3x^2$, взяв за точку старта $0.7$. Введите получившееся значение **с точностью до трёх знаков после точки-разделителя**.

Попробуйте реализовать алгоритм с использованием Python на основе алгоритма градиентного спуска, изученного в предыдущем модуле.

In [99]:
from sympy import Symbol, Eq, diff, solve

x = Symbol('x')
f = 6*x**5 - 5*x**4 - 4*x**3 + 3*x**2
print(len(solve(Eq(f,0))))
df = f.diff(x)

a_0 = 0.5
a_1 = 0
a_new = 0
i = 0
while True:
    if i > 10:
        break
    elif round(a_0,7) == round(a_1,7):
        print('stop')
        break
    a_new = a_0 - (f.subs(x,a_0)/df.subs(x,a_0))
    a_1 = a_0
    i += 1
    a_0 = a_new
    
    print(f'x= {round(a_new, 3)}, a_0= {a_0}, a_1 {a_1}')

4
x= 0.700, a_0= 0.700000000000000, a_1 0.5
x= 0.630, a_0= 0.629633507853403, a_1 0.700000000000000
x= 0.629, a_0= 0.628668078167331, a_1 0.629633507853403
x= 0.629, a_0= 0.628666978777900, a_1 0.628668078167331
x= 0.629, a_0= 0.628666978776461, a_1 0.628666978777900
stop


Отлично, мы научились искать приближённые значения для корней уравнения. Но как же это поможет нам найти **минимум** или **максимум** для функции?

Дело в том, что в задаче оптимизации можно решать не $f(x) = 0$, а $f'(x) = 0$ — тогда мы найдём потенциальные точки экстремума.

В многомерном случае по аналогичным рассуждениям производная превращается в градиент, а вторая производная превращается в гессиан (матрица вторых производных или, как мы её называли в предыдущем модуле, матрица Гессе). Поэтому в формуле появится обратная матрица.

Для многомерного случая формула выглядит следующим образом:

$x^{(n+1)} = x^{(n)} - \left [Hf(x^{(n)})  \right ]^{-1} \nabla f(x^{(n)})$

Можно заметить, что эта формула совпадает с формулой для градиентного спуска, но вместо умножения на `learning rate` (темп обучения) используется умножение на обратную матрицу к гессиану. Благодаря этому функция может сходиться за меньшее количество итераций, так как мы учитываем информацию о выпуклости функции через гессиан. Можно увидеть это на иллюстрации работы двух методов, где метод Ньютона явно сходится быстрее:

<img src=m6_img12.png width=900>

Метод Ньютона, если считать в количестве итераций, в многомерном случае (с гессианом) работает быстрее градиентного спуска.

Оптимизировать функцию $f(x) = x^3 - 3x^2 -45x  + 40$

Находим производную функции: $f^{\prime}=3 x^{2} - 6x - 45$

Находим вторую производную: $f^{\prime \prime}=6x - 6$

Сразу определим их в `Python`, чтобы можно было параллельно решить задачу и с помощью программирования:

In [92]:
def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6

Теперь необходимо взять какую-нибудь изначальную точку. Например, пусть это будет точка $x=42$. Также нам необходима точность — её возьмем равной $0.0001$. На каждом шаге будем переходить в следующую точку по уже упомянутой выше формуле:

$x^{(n+1)}=x^{(n)}-\frac{f^{\prime}\left(x^{(n)}\right)}{f^{\prime \prime}\left(x^{(n)}\right)}$

Например, в нашем случае следующая после $42$ точка будет рассчитываться следующим образом:

$x_{2}=42-\frac{f^{\prime}\left(x_{1}\right)}{f^{\prime \prime}\left(x_{2}\right)}$

$f^{\prime}\left(x_{1}\right)=3 x^{2}-6 x-45 \mid _{x_{1}=42}\;= 3 \cdot 42^{2}-6 \cdot 42-45=4995$

$f^{\prime \prime}\left(x_{1}\right)=6 x-6 \mid _{x_{1}=42} \; = \;6.42-6=246$

$x_{2}=42-\frac{4995}{246} \approx 42-20.305=21.695$

Третья точка будет вычисляться по аналогичному принципу:

$x_3 = 21.695 - \frac{f'(21.695)}{f''(21.695)}$

Но, к счастью, нам совсем не обязательно высчитывать всё вручную — воспользуемся Python и распишем наш алгоритм:

In [102]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

21.695121951219512
11.734125501243229
7.1123493600499685
5.365000391507974
5.015260627016227
5.000029000201801
5.000000000105126
5.000000000000001


Можно объединить всё в одну функцию:

In [103]:
def newtons_method(f, der, eps, init):
    iter_count = 0
    x_curr = init
    f = f(x_curr)
    while (abs(f) > eps):
        f = f(x_curr)
        f_der = der(x_curr)
        x_curr = x_curr - (f)/(f_prime)
        iter_count += 1
    return x_curr
 
from scipy.optimize import newton
newton(func=func1,x0=50,fprime=func2, tol=0.0001)


5.0

## Достоинства метода Ньютона

1. Если мы минимизируем квадратичную функцию, то с помощью метода Ньютона можно попасть в минимум целевой функции за один шаг.

2. Также этот алгоритм сходится за один шаг, если в качестве минимизируемой функции выступает функция из класса поверхностей вращения (т. е. такая, у которой есть симметрия).

3. Для несимметричной функции метод не может обеспечить сходимость, однако скорость сходимости  всё равно превышает скорость методов, основанных на градиентном спуске.

## Недостатки метода Ньютона

1. Этот метод **очень чувствителен к изначальным условиям**.

При использовании градиентного спуска мы всегда гарантированно движемся по антиградиенту в сторону минимума. В методе Ньютона происходит подгонка параболоида к локальной кривизне, и затем алгоритм движется к неподвижной точке данного параболоида. Из-за этого мы можем попасть в максимум или седловую точку. Особенно ярко это видно на невыпуклых функциях с большим количеством переменных, так как у таких функций седловые точки встречаются намного чаще экстремумов.

>Поэтому здесь необходимо обозначить ограничение: метод Ньютона стоит применять только для задач, в которых **целевая функция выпуклая**.

Впрочем, это не является проблемой. В линейной регрессии или при решении задачи классификации с помощью метода опорных векторов или логистической регрессии мы как раз ищем минимум у выпуклой целевой функции, то есть данный алгоритм подходит нам во многих случаях.

2. Также метод Ньютона может быть **затратным с точки зрения вычислительной сложности**, так как требует вычисления не только градиента, но и гессиана и обратного гессиана (при делении на матрицу необходимо искать обратную матрицу).

>Если у задачи много параметров, то расходы на память и время вычислений становятся астрономическими. Например, при наличии 50 параметров нужно вычислять более 1000 значений на каждом шаге, а затем предстоит ещё более 500 операций нахождения обратной матрицы. Однако метод всё равно используют, так как выгода от быстрой сходимости перевешивает затраты на вычисления.



Несмотря на его ограниченное практическое применение, метод Ньютона по-прежнему представляет большую ценность. Он имеет большое преимущество перед градиентным спуском **в силу своей быстроты и отсутствия необходимости в настройке гиперпараметра шага** (мы помним, что в градиентном спуске выбор шага — довольно непростая задача, а здесь можно обойтись без этого). Причём преимущество в быстроте очень ощутимое: в сравнении на реальных данных метод Ньютона находит решение задачи за 3 итерации, а градиентный спуск — за 489. То есть мы сильно выигрываем в скорости сходимости, а для анализа данных это очень важно, ведь экономия времени и вычислительных ресурсов позволяет решать задачи быстрее.

?Мы увидели, какой эффективной может быть оптимизация второго порядка при правильном использовании. Но что, если бы мы могли каким-то образом использовать эффективность, полученную при использовании производных второго порядка, но при этом избежать вычислительных затрат на вычисление обратного гессиана? Другими словами — можем ли мы создать алгоритм, который будет своего рода **гибридом между градиентным спуском и методом Ньютона**, где мы сможем получать более быструю сходимость, чем градиентный спуск, но меньшие вычислительные затраты на каждую итерацию, чем в методе Ньютона?

Оказывается, такой алгоритм существует. Точнее, целый класс таких методов оптимизации, называемых **квазиньютоновскими методами**.

**Задание 3.6**

Дана функция $f(x) = x^3 - 72x - 220$. Найдите решение уравнения $f(x)=0$ для поиска корня в окрестностях точки $x_0=12$.

In [106]:
initial_value = 12
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

def func1(x):
    return x**3 - 72*x - 220
def func2(x):
    return 3*x**2 - 72

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

10.21111111111111
9.756461564762278
9.727252176332618
9.72713442131889
9.727134419408875


**Задание 3.7**

Найдите положительный корень для уравнения $x^2 + 9x - 5 = 0$. В качестве стартовой точки возьмите $x_0 = 2.2$.

In [113]:
initial_value = 2.2
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

def func1(x):
    return x**2 - 9*x - 5
def func2(x):
    return 2*x - 9

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

-2.139130434782609
-0.7211696705674668
-0.5286253887576147
-0.5249391626429637
-0.524937810560627


**Задание 3.9**

С помощью метода Ньютона найдите точку минимума для функции $f(x) = 8x^3 - 2x^2 - 450$.

In [117]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

def func1(x):
    return 8*x**3-2*x**2-450
def func2(x):
    return 24*x**2-4*x

from scipy.optimize import newton
newton(func=func1,x0=42,fprime=func2, tol=0.0001)



3.916719200750909

# 4. Квазиньютоновские методы <a class="anchor" id=4></a>

[к содержанию](#0)

# 5. Линейное программирование <a class="anchor" id=5></a>

[к содержанию](#0)

# 6. Практика: Линейное программирование <a class="anchor" id=6></a>

[к содержанию](#0)

# 7. Практика: Дополнительные методы оптимизации <a class="anchor" id=7></a>

[к содержанию](#0)

# 8. Итоги <a class="anchor" id=8></a>

[к содержанию](#0)